# Refactored workflow example

This notebook runs the same history matching workflow as in the integration tutorial but uses the refactored history matcher that is currently in experimental as well as the GP emulator in experimental.

In [1]:
import torch

# imports from main
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard

# imports from experimental
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.history_matching import HistoryMatching

/Users/rjersakova/Documents/Projects/autoemulate/autoemulate/history_matching_dashboard.py:955: SyntaxWarning: invalid escape sequence '\m'
  return f"$\mu_{{{base_name}}}$"


## Simulate

Set up the Simulator and generate data OR read a data file if have run this previously.

In [2]:
parameters_range = extract_parameter_ranges(
    '../../../docs/data/naghavi_model_parameters.json'
)

simulator = NaghaviSimulator(
    parameters_range=parameters_range, 
    output_variables=['lv.P_i', 'lv.P_o'],  # Only the ones you're interested in
    n_cycles=300, 
    dt=0.001,
)

## Train a GP

(this should be done with AutoEmulate obviously)

In [25]:
gp_pytorch = GaussianProcessExact(
        x,
        y,
    )
gp_pytorch.fit(x, y)

## History Matching

Firstly, one can instantiate HistoryMatching without a simulator or an emulator. It can be used to calculate implausability for a given set of emulation predictions

In [7]:
# Define observed data with means and variances
observations = {
    'lv.P_i_min': (5.0, 0.1),   # Minimum of minimum LV pressure
    'lv.P_i_max': (20.0, 0.1),   # Maximum of minimum LV pressure
    'lv.P_i_mean': (10.0, 0.1),  # Mean of minimum LV pressure
    'lv.P_i_range': (15.0, 0.5), # Range of minimum LV pressure
    'lv.P_o_min': (1.0, 0.1),  # Minimum of maximum LV pressure
    'lv.P_o_max': (13.0, 0.1),  # Maximum of maximum LV pressure
    'lv.P_o_mean': (12.0, 0.1), # Mean of maximum LV pressure
    'lv.P_o_range': (20.0, 0.5)  # Range of maximum LV pressure
}

# Create history matcher
hm = HistoryMatching(
    observations=observations,
    threshold=3.0
)



History Matching involves:
- sampling parameters
- making predictions for those parameters
- evaluating implausability of predictions
- identifying which of the paraneters are not ruled out yet (NROY)

In [18]:
x = simulator.sample_inputs(5)
output = gp_pytorch.predict(torch.tensor(x, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)
implausability = hm.calculate_implausibility(pred_means, pred_vars)
nroy_indices = hm.get_nroy(implausability)
implausability, nroy_indices, x[nroy_indices]

(tensor([[0.5718, 6.1953, 1.6543, 4.7003, 2.3911, 3.0088, 2.5649, 7.1212],
         [0.5718, 6.1953, 1.6543, 4.7003, 2.3911, 3.0088, 2.5649, 7.1212],
         [0.5718, 6.1953, 1.6543, 4.7003, 2.3911, 3.0088, 2.5649, 7.1212],
         [0.5718, 6.1953, 1.6543, 4.7003, 2.3911, 3.0088, 2.5649, 7.1212],
         [0.5718, 6.1953, 1.6543, 4.7003, 2.3911, 3.0088, 2.5649, 7.1212]]),
 tensor([], dtype=torch.int64),
 array([], shape=(0, 16), dtype=float64))

There's also an option to directly filter inputs to get NROY parameters.

In [19]:
hm.filter_nroy_samples(x, pred_means, pred_vars)

array([], shape=(0, 16), dtype=float64)

Lastly, se can execture an iterative sample-predict-evaluate procedure with `HM.run()`. In each wave:
- sample parameter values to test from the NROY space
    - at the start, NROY is the entire parameter space
    - use emulator to filter out implausible samples
- make predictions for the sampled parameters using the simulator
- refit the emulator using the simulated data

In [22]:
simulator.in_dim = x.shape[1]
simulator.out_dim = y.shape[1]

tested_params, impl_scores, emulator = hm.run(
    n_waves=20,
    n_samples_per_wave=20,
    simulator=simulator,
    emulator=gp_pytorch,
)

History Matching:   0%|          | 0/20 [00:00<?, ?wave/s]


ValueError: Expected x to be TensorLike, got <class 'numpy.ndarray'>

In [11]:
tested_params.shape, impl_scores.shape

(torch.Size([400, 16]), torch.Size([400, 8]))

## Dashboard

In [ ]:
dashboard = HistoryMatchingDashboard(
    samples=tested_params,
    impl_scores=impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
    )

In [ ]:
dashboard.display()